<a href="https://colab.research.google.com/github/Roihan21/Iseng-/blob/main/face_detector_webcam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
from google.colab.patches import cv2_imshow  # Untuk menampilkan gambar di Colab

# Load the Haar Cascade for face and smile detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
smile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_smile.xml')

# Check if the Haar Cascades loaded successfully
if face_cascade.empty():
    print("Error: Could not load Haar Cascade for face detection.")
else:
    print("Haar Cascade for face detection loaded successfully.")

if smile_cascade.empty():
    print("Error: Could not load Haar Cascade for smile detection.")
else:
    print("Haar Cascade for smile detection loaded successfully.")

# Fungsi untuk mendeteksi wajah dan senyuman dari frame
def detect_faces_and_smiles(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Deteksi wajah
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Balok hijau untuk wajah

        # Region of Interest (ROI) untuk deteksi senyuman
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]

        # Deteksi senyuman dalam ROI
        smiles = smile_cascade.detectMultiScale(roi_gray, scaleFactor=1.8, minNeighbors=20, minSize=(25, 25))
        for (sx, sy, sw, sh) in smiles:
            cv2.rectangle(roi_color, (sx, sy), (sx + sw, sy + sh), (0, 0, 255), 2)  # Balok merah untuk senyuman

    return frame

# Menggunakan kamera di Google Colab
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

# Fungsi untuk mengambil gambar dari kamera
def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
            const div = document.createElement('div');
            const capture = document.createElement('button');
            capture.textContent = 'Capture';
            div.appendChild(capture);

            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});

            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();

            // Resize the output to fit the video element.
            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

            // Wait for Capture to be clicked.
            await new Promise((resolve) => capture.onclick = resolve);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            div.remove();
            return canvas.toDataURL('image/jpeg', quality);
        }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

# Ambil gambar dari kamera
filename = take_photo()

# Baca gambar yang diambil
frame = cv2.imread(filename)

# Deteksi wajah dan senyuman
frame_with_faces_and_smiles = detect_faces_and_smiles(frame)

# Tampilkan hasil
cv2_imshow(frame_with_faces_and_smiles)

Haar Cascade for face detection loaded successfully.
Haar Cascade for smile detection loaded successfully.


<IPython.core.display.Javascript object>

MessageError: NotAllowedError: Permission denied